In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [2]:
# Define dataset path
dataset_path = "E:/CP Dataset/cwt"

# Define image size
IMG_SIZE = 224

# Define categories (classes)
categories = ["Impeller (3.0BAR)", "Mechanical seal Hole (3BAR)", "Mechanical seal Scratch (3.0BAR)", "Normal (3BAR)"]
class_labels = {category: i for i, category in enumerate(categories)}  # Map class names to labels

# Load dataset
image_paths = []
labels = []

for category in categories:
    folder_path = os.path.join(dataset_path, category)
    for file in os.listdir(folder_path):
        if file.endswith(".png"):  # Ensure only images are loaded
            image_paths.append(os.path.join(folder_path, file))
            labels.append(class_labels[category])

# Split dataset into train (70%) and test (30%)
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.3, stratify=labels, random_state=42)

# Define image transformations (Resize + Normalize)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),  # Resize images
    transforms.ToTensor(),  # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

# Custom Dataset class
class CWDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.image_paths[idx], cv2.IMREAD_GRAYSCALE)  # Load grayscale image
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # Convert to RGB format
        
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)

        return img, label

# Create Dataloaders
batch_size = 32

train_dataset = CWDataset(train_paths, train_labels, transform=transform)
test_dataset = CWDataset(test_paths, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check dataset
print(f"Training samples: {len(train_dataset)}, Testing samples: {len(test_dataset)}")


Training samples: 872, Testing samples: 375


In [3]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)  # Adjusted for image size after convolutions
        self.fc2 = nn.Linear(512, num_classes)
        
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(x.size(0), -1)  # Flatten feature maps
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = SimpleCNN(num_classes=4).to(device)


Using device: cuda


In [6]:
# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
def train(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}, Accuracy: {train_acc:.2f}%")

# Train the model
train(model, train_loader, criterion, optimizer, epochs=10)


100%|██████████| 28/28 [01:13<00:00,  2.64s/it]


Epoch [1/10], Loss: 9.0486, Accuracy: 81.54%


100%|██████████| 28/28 [00:38<00:00,  1.39s/it]


Epoch [2/10], Loss: 8.5666, Accuracy: 83.72%


100%|██████████| 28/28 [00:38<00:00,  1.37s/it]


...

Epoch [8/10], Loss: 5.5453, Accuracy: 90.14%


100%|██████████| 28/28 [00:40<00:00,  1.43s/it]


Epoch [9/10], Loss: 5.8839, Accuracy: 91.63%


100%|██████████| 28/28 [00:59<00:00,  2.13s/it]

Epoch [10/10], Loss: 5.4704, Accuracy: 90.37%


In [7]:
# Evaluation Function
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Evaluate the model
evaluate(model, test_loader)


Test Accuracy: 90.13%
